<a href="https://colab.research.google.com/github/ElviiAi/useful-llm-notebooks/blob/main/tune-reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*More details in this article: [GRPO: Train LLMs with DeepSeek-R1's Reinforcement Learning Method](https://kaitchup.substack.com/p/grpo-train-llms-with-deepseek-r1s)*

This notebook shows how to train a GRPO adapter for LLMs. It uses Unsloth and a 14B Qwen2.5 model. It runs on a consumer GPU (24 GB) and also works on Google Colab's L4 GPU.

# Install

In [ ]:
!pip install unsloth vllm datasets diffusers
!pip install --upgrade pillow
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

  Using cached pillow-10.4.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
Using cached pillow-10.4.0-cp311-cp311-manylinux_2_28_x86_64.whl (4.5 MB)
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
  Using cached pillow-11.1.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
Using cached pillow-11.1.0-cp311-cp311-manylinux_2_28_x86_64.whl (4.5 MB)
  Attempting uninstall: pillow
    Found existing installation: pillow 10.4.0
    Uninstalling pillow-10.4.0:
      Successfully uninstalled pillow-10.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mistral-common 1.5.2 requires pillow<11.0.0,>=10.3.0, but you have pillow 11.1

# GRPO Pach for TRL

(I had to restart the notebook for this)

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-08 12:51:08 __init__.py:190] Automatically detected platform cuda.


# Load and Quantize Your Model for GRPO with QLoRA

In [ ]:
max_seq_length = 1024 # Should increase for longer reasoning traces
lora_rank = 16

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-32B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False to deactivate quantization
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.643 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-14b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.03%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 23.64 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.24 GB. Also swap space = 6 GB.
INFO 02-08 12:51:20 config.py:542] This model supports multiple tasks: {'reward', 'score', 'embed', 'generate', 'classify'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using k

[W208 12:51:21.678109817 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-08 12:51:21 loader.py:1102] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 02-08 12:51:22 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 02-08 12:51:27 model_runner.py:1115] Loading model weights took 10.5640 GB
INFO 02-08 12:51:27 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-08 12:51:30 worker.py:267] Memory profiling takes 2.13 seconds
INFO 02-08 12:51:30 worker.py:267] the current vLLM instance can use total_gpu_memory (23.64GiB) x gpu_memory_utilization (0.59) = 13.96GiB
INFO 02-08 12:51:30 worker.py:267] model weights take 10.56GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 0.88GiB; the rest of the memory reserved for KV Cache is 2.44GiB.
INFO 02-08 12:51:30 executor_base.py:110] # CUDA blocks: 831, # CPU blocks: 2048
INFO 02-08 12:51:30 executor_base.py:115] Maximum concurrency for 1024 tokens per request: 12.98x
INFO 02-08 12:51:35 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error o

Capturing CUDA graph shapes: 100%|██████████| 23/23 [00:19<00:00,  1.19it/s]

INFO 02-08 12:51:55 model_runner.py:1562] Graph capturing finished in 19 secs, took 0.84 GiB
INFO 02-08 12:51:55 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 27.26 seconds


# LoRA Configuration

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth 2025.2.5 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


# Reward Functions for GRPO

In [ ]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
<aha>
No no no, this is my real answer:
...
</aha>
"""
import re

def very_loose_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    responses = [completion[0]["content"] for completion in completions]
    return [0.5 if "<reasoning>" in r and "</reasoning>" in r else 0.0 for r in responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>.*?</reasoning>\s*<answer>.*?</answer>\s*<aha>.*?</aha>$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def aha_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion contains "aha" times, 2 for the tags, and one more, wherever it wants."""
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.findall(r'\baha\b', r, re.IGNORECASE) for r in responses]
    return [0.5 if len(match) == 3 else 0.0 for match in matches]

# Prompt Dataset

In [ ]:
from datasets import load_dataset
import multiprocessing
ds = load_dataset("cognitivecomputations/dolphin-r1", "reasoning-deepseek", split="train[:10000]")
ds = ds.rename_columns({'messages':'prompt'})

def process(row):
  row['prompt'][0]['content'] += '\n'+SYSTEM_PROMPT
  return row

ds= ds.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

def tokenize_with_template(example):
    """Tokenizes input text using the chat template of the tokenizer."""
    chat_template = tokenizer.apply_chat_template(example['prompt'], tokenize=False, add_generation_prompt=True)
    tokens = tokenizer(chat_template, truncation=False, add_special_tokens=True)["input_ids"]
    return {"token_length": len(tokens)}

ds = ds.map(tokenize_with_template,
    #num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False)
ds = ds.filter(lambda example: example["token_length"] <= max_seq_length)
ds = ds.remove_columns(["token_length"])

Map (num_proc=96):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
print(len(ds))

9917


# GRPO Configuration

In [ ]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 1e-6,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "paged_adamw_8bit",
    logging_steps = 5,
    bf16 = True,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory (but avoid below 4)
    max_prompt_length = 256,
    max_completion_length = 512,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


# GRPO Training

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        aha_reward_func,
        strict_format_reward_func,
        very_loose_format_reward_func
    ],
    args = training_args,
    train_dataset = ds,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,917 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 250
 "-____-"     Number of trainable parameters = 68,812,800


Step,Training Loss,reward,reward_std,completion_length,kl
5,0.000000,0.429167,0.069348,435.416669,0.000494
10,0.000000,0.462500,0.058005,408.366671,0.000879
15,0.000000,0.420833,0.030619,420.025008,0.000722
20,0.000000,0.354167,0.010206,414.283340,0.000717
25,0.000000,0.395833,0.047015,458.741673,0.000769
30,0.000000,0.425000,0.033322,423.000006,0.000717
35,0.000000,0.395833,0.036026,463.925006,0.000718
40,0.000000,0.429167,0.030619,458.033337,0.000760
45,0.000000,0.400000,0.074147,406.583340,0.000861
50,0.000000,0.408333,0.025820,396.391670,0.000776


TrainOutput(global_step=250, training_loss=3.0366615741513668e-05, metrics={'train_runtime': 33898.3931, 'train_samples_per_second': 0.029, 'train_steps_per_second': 0.007, 'total_flos': 0.0, 'train_loss': 3.0366615741513668e-05})

# Inference with the Trained Adapter

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/qwen2.5-14b-instruct-unsloth-bnb-4bit",
    device_map="cuda",
)

tokenizer = AutoTokenizer.from_pretrained("unsloth/qwen2.5-14b-instruct-unsloth-bnb-4bit")
model = PeftModel.from_pretrained(model, "outputs/checkpoint-250")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
<aha>
No no no, this is my real answer:
...
</aha>
"""

messages = [{"role":"system", "content":"You are an expert who knows the location of all zebras in America."+SYSTEM_PROMPT},
    {"role": "user", "content": "Make 10 possible equations to compute the number of zebras in America."},
]

seq = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(seq)

encoded_input = tokenizer(seq, return_tensors="pt").to(model.device)
outputs = model.generate(
  encoded_input['input_ids'],
  max_new_tokens=1024,
  do_sample=True,
  temperature=0.6,
  top_p=0.9,
)


response = outputs[0][encoded_input['input_ids'].shape[-1]:]
print(tokenizer.decode(response))

<|im_start|>system
You are an expert who knows the location of all zebras in America.
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
<aha>
No no no, this is my real answer:
...
</aha>
<|im_end|>
<|im_start|>user
Make 10 possible equations to compute the number of zebras in America.<|im_end|>
<|im_start|>assistant

<reasoning>
To compute the number of zebras in America, we need to consider various factors such as the population of different types of zoos, wildlife reserves, and private collections. Since actual numbers aren't provided, I'll create hypothetical equations based on these factors.

1. If we know the average number of zebras per zoo and the total number of zoos, we can multiply these values.
2. Similarly, if we know the number of wildlife reserves and the average number of zebras in each reserve, we can calculate the total.
3. We might also consider private collections, assuming we have data on the number of such collections and their av